In [3]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.lau

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-08-31 00:00:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [7]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [8]:
# Filter by price on certain columns
votes_over_20_df = vine_df.filter("total_votes>20")
votes_over_20_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
# create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = votes_over_20_df.withColumn("vote_usefulness", votes_over_20_df["helpful_votes"]/votes_over_20_df["total_votes"])

helpful_votes_df = helpful_votes_df.filter("vote_usefulness>=0.5")
helpful_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   vote_usefulness|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|  R39GSNW76GYF|          1|  

In [13]:
from pyspark.sql.functions import col

vine_paid = helpful_votes_df.filter(col("vine") == "Y")
vine_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   vote_usefulness|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|               0.9|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|               1.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|0.9285714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|0.9666666666666667|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|0.9615384615384616|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|              0.96|
|R25FI3J7WWOYXM|          3|           55|         58|   Y|                N|0.9482758620689655|
|R3P3U4EOWIUD50|          3|  

In [14]:
vine_not_paid = helpful_votes_df.filter(col("vine") == "N")
vine_not_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   vote_usefulness|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|  R39GSNW76GYF|          1|  

In [20]:
# Determine the total number of reviews, the number of 5-star reviews, and the 
# percentage of 5-star reviews for the two types of review (paid).

print(f"Reviews that were part of the Vine program:")
total_reviews_paid = vine_paid.count()
print(f"Total number of reviews: {total_reviews_paid}")

total_five_stars_paid = vine_paid.filter("star_rating == 5").count()
print(f"Total number of 5 star reviews: {total_five_stars_paid}")

review_percentage = (total_five_stars_paid/total_reviews_paid)*100
print(f"Percent of 5-star reviews:{review_percentage}%")


Reviews that were part of the Vine program:
Total number of reviews: 162
Total number of 5 star reviews: 63
Percent of 5-star reviews:38.88888888888889%


In [22]:
# Determine the total number of reviews, the number of 5-star reviews, and the 
# percentage of 5-star reviews for the two types of review (unpaid).

print(f"Reviews that were not part of the Vine program:")
total_reviews_not_paid = vine_not_paid.count()
print(f"Total number of reviews: {total_reviews_not_paid}")

total_five_stars_not_paid = vine_not_paid.filter("star_rating == 5").count()
print(f"Total number of 5 star reviews: {total_five_stars_not_paid}")

review_percentage_not_paid = (total_five_stars_not_paid/total_reviews_not_paid)*100
print(f"Percent of 5-star reviews:{review_percentage_not_paid}%")

Reviews that were not part of the Vine program:
Total number of reviews: 35606
Total number of 5 star reviews: 19444
Percent of 5-star reviews:54.60877380216817%


In [21]:
# Determine the total number of reviews, the number of 5-star reviews

total_reviews = helpful_votes_df.count()
print(f"Total number of reviews: {total_reviews}")

total_five_stars = helpful_votes_df.filter("star_rating == 5").count()
print(f"Total number of 5 star reviews: {total_five_stars}")

#review_percentage = (total_five_stars_paid/total_reviews_paid)*100
#print(f"Percent of 5-star reviews:{review_percentage}%")

Total number of reviews: 35768
Total number of 5 star reviews: 19507
